In [18]:
import requests
import urllib.request
import pandas as pd
import csv
from bs4 import BeautifulSoup


In [19]:

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"  # change to whatever your url is

response = requests.get(url)
print(response.status_code)


200


In [71]:
soup = BeautifulSoup(response.text,"html.parser")

table = soup.find('table',{"class":"wikitable sortable"})
tr = table.findAll(['tr'])[0:181]

In [79]:
op=[]
for i in range(1,len(tr)):
    tds = tr[i].find_all('td')
    l2=[tds[0].text.replace('\n',''), tds[1].text.replace('\n',''), tds[2].text.replace('\n','')]
    op.append(l2)

df=pd.DataFrame(op,columns=['Postal Code','Borough','Neighbourhood'])
df=df[df['Borough']!='Not assigned']


In [82]:
df.shape

(103, 3)